In [3]:
import pandas as pd

import streamlit as st
import pandas as pd
import os
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive



In [2]:
# Load Data
article = pd.read_csv("/Users/aswathshakthi/PycharmProjects/MNP25/data/articles.csv")
district = pd.read_csv("/Users/aswathshakthi/PycharmProjects/MNP25/data/District_beneficiary.csv")
public = pd.read_excel("/Users/aswathshakthi/PycharmProjects/MNP25/data/Public_Beneficiary_21_24.xlsx")



article_name = article["Articles"].unique().tolist()

if article_name:
    for art in article_name:

        cpu = article[article["Articles"] == art]["Cost per unit"].tolist()[0]
        print(cpu)


100000.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
5000.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
10000.0
20500.0
20500.0
20500.0
20500.0
5000.0


In [3]:
article

,Articles,Cost per unit,Item Type
0,Handicapped Hand Tricycle,100000.0,Article
1,Fixed Wheel Chair,20500.0,Article
2,Wheelchair,20500.0,Article
3,Handicapped Scooter,20500.0,Article
4,Two Wheeler,20500.0,Article
...,...,...,...
65,Sewing Machine Heavy,20500.0,Article
66,Sewing Machine ORD / Motor,20500.0,Article
67,Sewing Machine ORD,20500.0,Article
68,Gents Cycle,20500.0,Article


In [4]:
article

,Articles,Cost per unit,Item Type
0,Handicapped Hand Tricycle,100000.0,Article
1,Fixed Wheel Chair,20500.0,Article
2,Wheelchair,20500.0,Article
3,Handicapped Scooter,20500.0,Article
4,Two Wheeler,20500.0,Article
...,...,...,...
65,Sewing Machine Heavy,20500.0,Article
66,Sewing Machine ORD / Motor,20500.0,Article
67,Sewing Machine ORD,20500.0,Article
68,Gents Cycle,20500.0,Article


In [7]:
import io
import json
import streamlit as st
import pandas as pd
import os
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from googleapiclient.http import MediaFileUpload, MediaIoBaseUpload, MediaIoBaseDownload
from googleapiclient.discovery import build
from google.oauth2 import service_account
import streamlit_authenticator as stauth
import yaml
import time
from yaml.loader import SafeLoader

master_data_id = "1ry614-7R4-s0uQcv0zrNeS4O0KAbhVEC67rl5_VllGI"  # Replace with your file's ID
article_data_id = "1b7eyqlN3lTapBRYcO1VrXGsj_gBVSxQLIyLCPu3UcG8"
district_data_id = "1lwJL-_KQaOY3VSd2cOeOdiR5QOn8yvX3zp6xNfQJo9U"
public_data_id = "1sO08BfwN1gzNs_N7XDq1RnqMgJDDKMdq_nsaNhmjKhs"

## Access API from local json
creds = service_account.Credentials.from_service_account_file('/Users/aswathshakthi/PycharmProjects/MNP25/data/mnpdatabase-ca1a93fefdd6.json',
        scopes=['https://www.googleapis.com/auth/drive'])

## Access via streamlit secrets
credentials_dict = json.loads(st.secrets["gcp"]["credentials"])
# creds = service_account.Credentials.from_service_account_info(credentials_dict, scopes=['https://www.googleapis.com/auth/drive'])
drive_service = build('drive', 'v3', credentials=creds)


In [8]:
def read_file(file_id):
    try:
        # Specify the desired MIME type for export (e.g., text/csv for Sheets)
        request = drive_service.files().export_media(
            fileId=file_id,
            mimeType='text/csv'  # Change this based on the file type
        )
        file_stream = io.BytesIO()
        downloader = MediaIoBaseDownload(file_stream, request)

        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Download progress: {int(status.progress() * 100)}%")

        file_stream.seek(0)  # Reset the stream position
        df = pd.read_csv(file_stream)
        return df

    except Exception as e:
        st.error(f"Failed to read file: {e}")

In [9]:


def update_file(file_id, updated_df):
    updated_stream = io.BytesIO()
    updated_df.to_csv(updated_stream, index=False)
    updated_stream.seek(0)

    media = MediaIoBaseUpload(updated_stream, mimetype="text/csv")
    updated_file = drive_service.files().update(
        fileId=file_id,
        media_body=media).execute()

    alert1 = st.success(f"File updated: {updated_file.get('id')}")
    time.sleep(1)
    alert1.empty()

In [10]:
public_master_id = "1EdEySmYe6ZJUW16f65_q30nkqfbvDADjcmEkAEJrrL4"

public = read_file(public_data_id)
public_master = read_file(public_master_id)
public_master

Download progress: 100%
Download progress: 100%


,App. No.,Name,Aadhar (Without Space),Handicapped (Yes / No),Address,Mobile,Article Name,Comments


In [13]:
district = read_file(master_data_id)
district

Download progress: 100%


,NAME OF THE DISTRICT,REQUESTED ARTICLE,ITEM TYPE,QUANTITY,COST PER UNIT,TOTAL COST,ALLOTTED FUNDS,EXCESS/SHORTAGE,COMMENTS
0,Andaman,Handicapped Aid,Aid,1,5000.0,5000.0,10000.0,NaN,Public
1,Andaman,Handicapped Aid,Aid,1,5000.0,5000.0,NaN,0.0,Private
2,Chengalpattu,Agri Power Sprayer (1.2 Bolo),Article,2,11760.0,23520.0,160000.0,NaN,NaN
3,Chengalpattu,Gents Cycle,Article,8,5500.0,44000.0,NaN,NaN,NaN
4,Chengalpattu,Girls Cycle,Article,8,5200.0,41600.0,NaN,NaN,NaN
5,Chengalpattu,Iron Box,Article,8,7000.0,56000.0,NaN,NaN,NaN
6,Chengalpattu,Sewing Machine ORD,Article,6,5200.0,31200.0,NaN,-36320.0,NaN
7,Krishnagiri-B3,Audio Mixer,Article,1,NaN,NaN,52250.0,52250.0,NaN
8,Salem,Girls Cycle,Article,1,5200.0,5200.0,600000.0,NaN,NaN
9,Salem,Iron Box,Article,2,7000.0,14000.0,NaN,NaN,NaN


In [15]:
saved_data = read_file(master_data_id)
existing_record = saved_data[
                    (saved_data["NAME OF THE DISTRICT"] == "Andaman") &
                    (saved_data["REQUESTED ARTICLE"] == "Handicapped Aid")]

Download progress: 100%


In [16]:
existing_record

,NAME OF THE DISTRICT,REQUESTED ARTICLE,ITEM TYPE,QUANTITY,COST PER UNIT,TOTAL COST,ALLOTTED FUNDS,EXCESS/SHORTAGE,COMMENTS
0,Andaman,Handicapped Aid,Aid,1,5000.0,5000.0,10000.0,NaN,Public
1,Andaman,Handicapped Aid,Aid,1,5000.0,5000.0,NaN,0.0,Private


In [19]:
existing_record["QUANTITY"].values[0]

np.int64(1)